In [1]:
# n_aug = 0,1,2,4,8,16,32
# N = 500,2000,5000,Full
# 


In [2]:
from methods import *
import os, shutil

In [3]:
#user inputs

#load hyperparameters
sizes = ['1_tiny', '2_small', '3_standard', '4_full']
size_folders = ['naug_sized_f1/' + size for size in sizes]

#datasets
datasets = ['sst2']

#number of output classes
num_classes_list = [2, 2, 2, 6, 2]

#number of augmentations per original sentence
n_aug_list_dict = {'size_data_f1/1_tiny': [32, 32, 32, 32, 32], 
					'size_data_f1/2_small': [32, 32, 32, 32, 32],
					'size_data_f1/3_standard': [16, 16, 16, 16, 4],
					'size_data_f1/4_full': [16, 16, 16, 16, 4]}

if not os.path.isdir('size_data_f1'):
    os.mkdir('size_data_f1')
#number of words for input
input_size_list = [50, 50, 40, 25, 25]

alphas = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

# Number of words for input
input_size_list = [50,50,40,25,25]

#word2vec dictionary
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300

In [4]:
def run_cnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_cnn(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=20, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=1)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [7]:
#for each method
a_method = 'weda' #### TOGGLE EDA/WEDA
tfidf_func = 0
n_augs = [16,32]

for n_aug in n_augs:

    writer_cnn = open('outputs_naug/cnn_' + a_method + '_' +'tfidf'+str(tfidf_func)+'_'+'naug'+str(n_aug) +'_'+ get_now_str() + '.txt', 'w')

    #for storing the performances
    performances_cnn = {size_folder:[] for size_folder in size_folders}

    #for each percentage dataset
    for size_folder in size_folders:

        writer_cnn.write(size_folder + '\n')

        #get all six datasets
        dataset_folders = [size_folder + '/' + s for s in datasets]



        #for each dataset
        for i in range(len(dataset_folders)):

            #initialize all the variables
            dataset_folder = dataset_folders[i]
            dataset = datasets[i]
            num_classes = num_classes_list[i]
            input_size = input_size_list[i]
            word2vec_pickle = dataset + '/word2vec.p'
            word2vec = load_pickle(word2vec_pickle)



            train_path = dataset_folder + '/train_weda_naug_'+str(n_aug)+'.txt' #### TOGGLE EDA/WEDA
            test_path = dataset + '/test.txt'
            acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
            print("cnn aug ", acc, dataset_folder)

            performances_cnn[size_folder].append(acc)
            """
            acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
            performances_rnn[alpha].append(acc)
            print("rnn aug", acc)
            """

    writer_cnn.write(str(performances_cnn) + '\n')
    #writer_rnn.write(str(performances_rnn)+"\n")
    for size_folder in performances_cnn:
        line = str(size_folder) + ' : ' + str(sum(performances_cnn[size_folder])/len(performances_cnn[size_folder]))
        writer_cnn.write(line + '\n')
        print(line)
    """
    for alpha in performances_rnn:
        line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
        writer_rnn.write(line + '\n')
        print(line)
    """
    print(performances_cnn)

    writer_cnn.close()    


Epoch 1/20
8/8 [==============================] - 2s 183ms/step - loss: 0.6807 - accuracy: 0.5795 - val_loss: 0.6168 - val_accuracy: 0.8059
Epoch 2/20
8/8 [==============================] - 1s 152ms/step - loss: 0.5831 - accuracy: 0.8488 - val_loss: 0.5071 - val_accuracy: 0.8353
Epoch 3/20
8/8 [==============================] - 1s 155ms/step - loss: 0.4550 - accuracy: 0.8796 - val_loss: 0.3797 - val_accuracy: 0.8718
Epoch 4/20
8/8 [==============================] - 1s 155ms/step - loss: 0.3168 - accuracy: 0.9183 - val_loss: 0.2675 - val_accuracy: 0.9212
Epoch 5/20
8/8 [==============================] - 1s 157ms/step - loss: 0.2026 - accuracy: 0.9480 - val_loss: 0.1754 - val_accuracy: 0.9424
Epoch 6/20
8/8 [==============================] - 1s 153ms/step - loss: 0.1171 - accuracy: 0.9800 - val_loss: 0.1061 - val_accuracy: 0.9765
Epoch 7/20
8/8 [==============================] - 1s 177ms/step - loss: 0.0621 - accuracy: 0.9936 - val_loss: 0.0657 - val_accuracy: 0.9847
Epoch 8/20
8/8 [====

75/75 [==============================] - 12s 157ms/step - loss: 4.3454e-04 - accuracy: 1.0000 - val_loss: 0.0370 - val_accuracy: 0.9885
Epoch 18/20
75/75 [==============================] - 11s 152ms/step - loss: 4.4768e-04 - accuracy: 0.9999 - val_loss: 0.0370 - val_accuracy: 0.9886
cnn aug  0.8273299028016009 naug_sized_f1/3_standard/sst2
Epoch 1/20
111/111 [==============================] - 28s 237ms/step - loss: 0.5075 - accuracy: 0.7465 - val_loss: 0.2698 - val_accuracy: 0.8925
Epoch 2/20
111/111 [==============================] - 23s 209ms/step - loss: 0.1979 - accuracy: 0.9330 - val_loss: 0.1023 - val_accuracy: 0.9683
Epoch 3/20
111/111 [==============================] - 20s 181ms/step - loss: 0.0541 - accuracy: 0.9893 - val_loss: 0.0618 - val_accuracy: 0.9783
Epoch 4/20
111/111 [==============================] - 19s 169ms/step - loss: 0.0182 - accuracy: 0.9979 - val_loss: 0.0501 - val_accuracy: 0.9837
Epoch 5/20
111/111 [==============================] - 19s 167ms/step - loss: 0

cnn aug  0.8096054888507719 naug_sized_f1/2_small/sst2
Epoch 1/20
146/146 [==============================] - 38s 247ms/step - loss: 0.4611 - accuracy: 0.7858 - val_loss: 0.1318 - val_accuracy: 0.9615
Epoch 2/20
146/146 [==============================] - 29s 196ms/step - loss: 0.0868 - accuracy: 0.9773 - val_loss: 0.0728 - val_accuracy: 0.9755
Epoch 3/20
146/146 [==============================] - 27s 185ms/step - loss: 0.0228 - accuracy: 0.9965 - val_loss: 0.0444 - val_accuracy: 0.9848
Epoch 4/20
146/146 [==============================] - 26s 177ms/step - loss: 0.0095 - accuracy: 0.9989 - val_loss: 0.0244 - val_accuracy: 0.9924
Epoch 5/20
146/146 [==============================] - 29s 197ms/step - loss: 0.0033 - accuracy: 0.9997 - val_loss: 0.0217 - val_accuracy: 0.9933
Epoch 6/20
146/146 [==============================] - 27s 183ms/step - loss: 0.0018 - accuracy: 0.9999 - val_loss: 0.0212 - val_accuracy: 0.9935
Epoch 7/20
146/146 [==============================] - 27s 185ms/step - loss